In [2]:
import configparser as parser
import pymysql
import pandas as pd
import numpy as np


props = parser.ConfigParser()
props.read("./../config.ini")
conf = props

In [3]:
# 테이블 생성
create_table_sql = """
CREATE TABLE IF NOT EXISTS oil_test (
    시도 VARCHAR(50),
    시군구 VARCHAR(50),
    이름 VARCHAR(100),
    셀프 VARCHAR(10),
    주소 VARCHAR(255),
    번호 VARCHAR(20),
    고급휘발유 INT,
    보통휘발유 INT,
    경유 INT,
    실내등유 INT,
    PRIMARY KEY (시군구, 이름, 주소, 번호)
);
"""

with pymysql.connect(host=conf['MYSQL']['host'], port=3306, user=conf['MYSQL']['user'], password=conf['MYSQL']['password'], db=conf['MYSQL']['db']) as conn:
    with conn.cursor() as cur:
        cur.execute(create_table_sql)
        conn.commit()

In [6]:
# csv > mysql
df = pd.read_csv("./../oil_data.csv", encoding='cp949')

df['고급휘발유'] = df['고급휘발유'].replace(np.nan, None)
df['보통휘발유'] = df['보통휘발유'].replace(np.nan, None)
df['경유'] = df['경유'].replace(np.nan, None)
df['실내등유'] = df['실내등유'].replace(np.nan, None)

with pymysql.connect(host=conf['MYSQL']['host'], port=3306, user=conf['MYSQL']['user'], password=conf['MYSQL']['password'], db=conf['MYSQL']['db']) as conn:
    with conn.cursor() as cur:
        insert_sql = """
        INSERT INTO oil_test (시도, 시군구, 이름, 셀프, 주소, 번호, 고급휘발유, 보통휘발유, 경유, 실내등유)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Insert each row into the MySQL table
        for index, row in df.iterrows():
            cur.execute(insert_sql, tuple(row))

        # Commit the transaction
        conn.commit()

In [4]:
# mysql > csv
with pymysql.connect(host=conf['MYSQL']['host'], port=3306, user=conf['MYSQL']['user'], password=conf['MYSQL']['password'], db=conf['MYSQL']['db']) as conn:
    with conn.cursor() as cur:
        sql = "SELECT * FROM oil_data"
        cur.execute(sql)
        rows = cur.fetchall()
        columns = [col[0] for col in cur.description]
    
df = pd.DataFrame(rows, columns=columns)
df.to_csv("./../oil_data.csv", index=False, encoding='cp949')